In [1]:
import requests                 # requests on websites
from bs4 import BeautifulSoup   # html parsing
import pandas as pd             # pandas for data frame
import math                     # for math methods
import time                     # for sleep timer
import random
from sql_functions import *     # functions from file for upload on schema
import psycopg2                 # for upload on engine
import datetime as dt           # for the csv file with the current date and time
import re

In [2]:
engine = get_engine()

schema = 'capstone_jmrs'

In [3]:
sql_text = f"""
CREATE TABLE capstone_jmrs.spotahome_merged AS
SELECT *
FROM capstone_jmrs.spotahome_df_complete_available_from sdcaf
LEFT JOIN capstone_jmrs.spotahome_df_details_complete sddc  
	   ON sdcaf.platform_id = sddc.id;
"""
result = engine.execute(sql_text)

In [4]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
	SET available_from_2 = COALESCE(available_from_2, '2022');
"""
result = engine.execute(sql_text)

In [5]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
	ADD available_from_total VARCHAR;"""
result = engine.execute(sql_text)

In [6]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged 
	SET available_from_total = available_from_0 || '-' || available_from_1 || '-' || available_from_2;
"""
result = engine.execute(sql_text)

In [7]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
    ALTER COLUMN platform_id TYPE VARCHAR,
    ALTER COLUMN platform TYPE VARCHAR,
    ALTER COLUMN neighborhood TYPE VARCHAR,
    ALTER COLUMN housing_type TYPE VARCHAR,
    ALTER COLUMN property_type TYPE VARCHAR,
    ALTER COLUMN title TYPE VARCHAR,
    ALTER COLUMN furnished TYPE VARCHAR,
    ALTER COLUMN let_type TYPE VARCHAR,
    --available_today
    ALTER COLUMN scraping_date TYPE DATE USING scraping_date::DATE,
    ALTER COLUMN price_pcm_0 TYPE FLOAT USING price_pcm_0::FLOAT,
    ALTER COLUMN price_pcm_1 TYPE FLOAT USING price_pcm_1::FLOAT,
    ALTER COLUMN bathrooms TYPE FLOAT USING bathrooms::FLOAT,
    ALTER COLUMN m2 TYPE FLOAT USING m2::FLOAT,
    ALTER COLUMN bedrooms TYPE FLOAT USING bedrooms::FLOAT,
    ALTER COLUMN available_from_total TYPE DATE USING available_from_total::DATE;
"""
result = engine.execute(sql_text)

In [8]:
sql_text = f"""   
ALTER TABLE capstone_jmrs.spotahome_merged
  DROP COLUMN available_from,
  DROP COLUMN available_from_0,
  DROP COLUMN available_from_1,
  DROP COLUMN available_from_2,
  DROP COLUMN id;
"""
result = engine.execute(sql_text)

In [9]:
sql_text = f""" 
UPDATE capstone_jmrs.spotahome_merged
SET m2 = NULL
WHERE m2 = '1'
	OR m2 = '3'
	OR m2 = '8'
	OR m2 = '1000';
"""
result = engine.execute(sql_text)

In [10]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
	RENAME COLUMN available_from_total TO available_from;
"""
result = engine.execute(sql_text)

In [11]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
	RENAME COLUMN furnished TO furniture;
"""
result = engine.execute(sql_text)

In [12]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
	RENAME COLUMN m2 TO size_sqm;
"""
result = engine.execute(sql_text)

In [13]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
	RENAME COLUMN neighborhood TO neighbourhood;
"""
result = engine.execute(sql_text)

In [14]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET bedrooms = COALESCE(bedrooms, '0')
WHERE housing_type = 'studios';
"""
result = engine.execute(sql_text)

In [15]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET bedrooms = COALESCE(bedrooms, '1')
WHERE housing_type = 'apartments/bedrooms:1';
"""
result = engine.execute(sql_text)

In [16]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET bedrooms = COALESCE(bedrooms, '2')
WHERE housing_type = 'apartments/bedrooms:2';
"""
result = engine.execute(sql_text)

In [17]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET bedrooms = COALESCE(bedrooms, '3')
WHERE housing_type = 'apartments/bedrooms:3';
"""
result = engine.execute(sql_text)

In [18]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET bedrooms = COALESCE(bedrooms, '4')
WHERE housing_type = 'apartments/bedrooms:3more';
"""
result = engine.execute(sql_text)

In [19]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
	DROP COLUMN housing_type;
"""
result = engine.execute(sql_text)

In [20]:
sql_text = f"""
 UPDATE capstone_jmrs.spotahome_merged
	SET available_today = CASE 
      						WHEN available_from = CURRENT_DATE  THEN 'available'
      						ELSE 'occupied'
						  END;
"""
result = engine.execute(sql_text)